# Find possible client duplicates using the Levenshtein Edit Distance

In [ ]:
# Install a conda package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install python-Levenshtein

In [9]:
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import GuestbookDbConnect as gdb
import Levenshtein

conn = gdb.guestbookDbConnect()

Welcome to Jupyter Notebook.  You are connected to the Opportunity House guestbook database!


In [11]:
# if both of the following are 0, then exact matches only will be found
maxFirstDelta = 2
maxLastDelta  = 2

person_query = "select idperson, firstname, lastname \
         from guestbook_person \
         order by idperson"

#print(person_query)

data = pd.read_sql(person_query, conn)

#data.head(100)

clientlist = []

for n in range(0, data.shape[0]):
#for n in range(0, 60):  
  id = data.iloc[n,0] 
  lastname  = data.iloc[n,2]
  firstname = data.iloc[n,1]
  client = {}
  client['id'] = id 
  client['first'] = firstname
  client['last'] = lastname
  client['dups'] = []
  #print("{} {}".format(lastname, firstname))
  for m in range(0, data.shape[0]):
    if m!=n:
      cid = data.iloc[m,0] 
      clastname  = data.iloc[m,2]
      cfirstname = data.iloc[m,1]
      last  = Levenshtein.distance(lastname, clastname)
      first = Levenshtein.distance(firstname, cfirstname)
      if (last<=maxLastDelta) and (first<=maxFirstDelta):
        dup = {}
        dup['id'] = cid 
        dup['first'] = cfirstname
        dup['last'] = clastname
        #print("  {} {}".format(clastname, cfirstname))
        client['dups'].append(dup)
  if (len(client['dups'])>0):
    clientlist.append(client)  
#print(clientlist)


In [12]:
for client in clientlist:
    print("\n{}..{}..{}".format(client['id'], client['first'], client['last']))
    for duplicate in client['dups']:
        print("   {}..{}..{}".format(duplicate['id'], duplicate['first'], duplicate['last']))


20..JOHN..WAYNE
   721..JOHN..WAYNE

44..CLAYTON..RICHARDS
   1228..CLAYTON..RICHA4DS

49..KATIE..SMALLEY
   1580..JAMIE..TALLEY

51..DOT..CARR
   280..JOC..EARL

54..PATRICIA ..REID
   234..PATRICIA..REID
   1064..PATRICIA..READ

56..BRANDON..AGUILAR
   1171..BRANDON..AGILAR

61..SHANA..NOLL
   690..SHAWN..GILL
   1363..SHANA..NOLL

68..DAVE..WALKER
   803..JADE..PARKER
   1436..GAGE..WALTHER

69..GERALD..STEIN
   1600..JERALD..STIEN

72..JAMES..BAILEY
   1580..JAMIE..TALLEY

76..LARRY..LANE
   1227..LARRY..WADE

82..RECO..REID
   1563..RICO..REID

89..LINDA..NELSON
   970..BELINDA..LETSON

90..BJ..ISOM
   723..DJ..ISOM

94..GEORGE..THOMPSAN
   1091..GEORGE..THOMPSON

99..RICK..HAMES
   614..RICKY..HAMES

108..BOBBY..BARTON
   1110..BOBBY..MARTIN

159..AUSTRALIA..ROGERS
   1538..AUSTRALIA..ROGERS

160..DEJAY..JAMES
   857..JAY..DAMMS

188..DONNIE ..VINCENT
   674..DONNIE..VINCENT

200..BRANDON..PLESS
   725..BRANDON..PLESS
   850..BRANDON..PLESS

211..LARRY..MASON
   291..BARRY..MASO

In [8]:
person_query = "select idperson, firstname, lastname \
  from guestbook_person \
  where (firstname, lastname) in ( \
     select firstname, lastname \
       from guestbook_person \
       group by firstname, lastname \
       having count(*) > 1 \
     ) \
  order by lastname, firstname" 

print(person_query)

data = pd.read_sql(person_query, conn)

#data.head(100)

for n in range(0, data.shape[0]):  
  id = data.iloc[n,0] 
  lastname  = data.iloc[n,2]
  firstname = data.iloc[n,1]  
  print("{} {}\n".format(id,lastname,firstname))

select idperson, firstname, lastname   from guestbook_person   where (firstname, lastname) in (      select firstname, lastname        from guestbook_person        group by firstname, lastname        having count(*) > 1      )   order by lastname, firstname
505 ARCHER

482 ARCHER

506 ARCHER

483 ARCHER

1533 BARRETT

1534 BARRETT

1616 CALL

1617 CALL

1463 CHAMBERS

1464 CHAMBERS

1465 CHAMBERS

405 COOPER

238 COOPER

1657 CRAWFORD

1656 CRAWFORD

1296 DANIELS

1297 DANIELS

1321 EARNEST

662 EARNEST

643 FRAZIER

1586 FRAZIER

323 GAY

326 GAY

1342 HOLBROOK

745 HOLBROOK

1354 JOHNSON

399 JOHNSON

1061 JONES

322 JONES

1622 KING

1623 KING

1315 LICARE

299 LICARE

1174 LUX

1077 LUX

1626 MCLEAN

1625 MCLEAN

421 MITCHELL

267 MITCHELL

1609 MOORE

1608 MOORE

440 MORGAN

246 MORGAN

1181 MORRIS

1640 MORRIS

61 NOLL

1363 NOLL

566 PARKS

713 PARKS

1515 PATTERSON

1191 PATTERSON

965 PEARSON

274 PEARSON

200 PLESS

725 PLESS

850 PLESS

470 RED

469 RED

1396 RICHARDSON

139